# Deploy Llama 3.1 405B on Google Cloud Vertex AI with Text Generation Inference

[Llama 3.1](https://huggingface.co/blog/llama31) is the latest is the open LLM from Meta, a follow up iteration of Llama 3, released in July 2024. Llama 3.1 comes in three sizes: 8B for efficient deployment and development on consumer-size GPU, 70B for large-scale AI native applications, and 405B for synthetic data, LLM as a Judge or distillation; among other use cases. Amongst Llama 3.1 new features we highlight the following:a large context length of 128K tokens (vs original 8K), multilingual capabilities, tool usage capabilities, and a more permissive license.

In this blog you will learn how to deploy [`meta-llama/Meta-Llama-3.1-405B-Instruct-FP8`](https://hf.co/meta-llama/Meta-Llama-3.1-405B-Instruct-FP8) in a node with 8 x H100 NVIDIA GPUs on Google Vertex AI with Text Generation Inference (TGI) via the new purpose-built Deep Learning Container (DLC) for Google Cloud.

This blog will cover how to:

1. [Setup Google Cloud for Vertex AI](#1-setup-google-cloud-for-vertex-ai)
2. [Register Llama 3.1 405B on Vertex AI](#2-register-llama-31-405b-on-vertex-ai)
3. [Deploy Llama 3.1 405B on Vertex AI](#3-deploy-llama-31-405b-on-vertex-ai)
4. [Run online predictions with Llama 3.1 405B](#4-run-online-predictions-with-llama-31-405b)
5. [Clean up resources](#5-clean-up-resources)

Lets get started! 🚀

## Introduction to Vertex AI

Vertex AI is a machine learning (ML) platform that lets you train and deploy ML models and AI applications, and customize large language models (LLMs) for use in your AI-powered applications. Vertex AI combines data engineering, data science, and ML engineering workflows, enabling your teams to collaborate using a common toolset and scale your applications using the benefits of Google Cloud.

This blog will be focused on deploying an already fine-tuned model from the Hugging Face Hub using prebuilt container to get real-time online predictions.

More information at [Vertex AI - Documentation - Introduction to Vertex AI](https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform).

## 1. Setup Google Cloud for Vertex AI

Before proceeding, for convenience we will set the following environment variables:

In [ ]:
%env PROJECT_ID=your-project-id
%env LOCATION=your-region

First you need to install `gcloud` in your machine following the instructions at <https://cloud.google.com/sdk/docs/install>; and log in into your Google Cloud account, setting your project and preferred Google Compute Engine region.

In [ ]:
%%bash
gcloud auth login
gcloud config set project $PROJECT_ID
gcloud config set compute/region $LOCATION

Once the Google Cloud SDK is installed, you need to enable the Google Cloud APIs required to use Vertex AI from a Deep Learning Container (DLC) within their Artifact Registry for Docker.

In [ ]:
%%bash
gcloud services enable aiplatform.googleapis.com
gcloud services enable compute.googleapis.com
gcloud services enable container.googleapis.com
gcloud services enable containerregistry.googleapis.com
gcloud services enable containerfilesystem.googleapis.com

Then you will also need to install `google-cloud-aiplatform`, required to programmatically interact with Google Cloud Vertex AI from Python.

In [ ]:
%%bash
pip install --upgrade --quiet google-cloud-aiplatform

In [ ]:
import os
from google.cloud import aiplatform

aiplatform.init(project=os.getenv("PROJECT_ID"), location=os.getenv("LOCATION"))

Finally, since [`meta-llama/Meta-Llama-3.1-405B-Instruct-FP8`](https://hf.co/meta-llama/Meta-Llama-3.1-405B-Instruct-FP8) is a gated model in the Hugging Face Hub, you will also need to install `huggingface_hub` to use the `huggingface-cli` to log in into the Hugging Face Hub.

In [ ]:
%%bash
pip install --upgrade --quiet huggingface_hub
huggingface-cli login

Alternatively, you can also skip the `huggingface_hub` installation and just generate a [Hugging Face Fine-grained Token](https://hf.co/settings/tokens) with read-only permissions for [`meta-llama/Meta-Llama-3.1-405B-Instruct-FP8`](https://hf.co/meta-llama/Meta-Llama-3.1-405B-Instruct-FP8) selected under `Repository permissions -> meta-llama/Meta-Llama-3.1-405B-Instruct-FP8 -> Read access to contents of selected repos`.

In [ ]:
import os
from pathlib import Path

HF_TOKEN = os.getenv("HF_TOKEN", open(str(Path.home() / ".cache/huggingface/token")).read())

## 2. Register Llama 3.1 405B on Vertex AI

To register Llama 3.1 405B on Vertex AI, you will need to use the `google-cloud-aiplatform` Python SDK. But before proceeding you need to first define which DLC are you going to use, which in this case will be the latest Hugging Face TGI DLC for GPU.

As of the current date, July 2024, the latest available Hugging Face TGI DLC is us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204 which uses TGI v2.2 that comes with support for the Llama 3.1 architecture as it needs a custom RoPE scaling method than its predecessor, Llama 3 (and 2).

To check which Hugging Face DLCs are available in Google Cloud you can either navigate to [Google Cloud Artifact Registry](https://console.cloud.google.com/artifacts/docker/deeplearning-platform-release/us/gcr.io) and filter by "huggingface-text-generation-inference", or use the following `gcloud` command:

In [ ]:
%%bash
gcloud container images list --repository="us-docker.pkg.dev/deeplearning-platform-release/gcr.io" | grep "huggingface-text-generation-inference"

Then you need to define the configuration for the container, which are the environment variables that the `text-generation-launcher` expects as arguments (as per the [official documentation](https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/launcher)), which in this case are the following:

* `MODEL_ID` the model ID on the Hugging Face Hub, i.e. `meta-llama/Meta-Llama-3.1-405B-Instruct-FP8`.
* `REVISION` the model revision on the Hugging Face Hub which can be a branch name or a commit hash, i.e. `7c365a34d0fed5b4afb118cf253395f47f6efa72`, most recent to current date and the tested extensively within this blog.
* `HUGGING_FACE_HUB_TOKEN` the read-access token over the gated repository `meta-llama/Meta-Llama-3.1-405B-Instruct-FP8`, required to download the weights from the Hugging Face Hub.
* `NUM_SHARD` the number of shards to use i.e. the number of GPUs to use, in this case set to 8 as a node with 8 x H100s will be used.
* `QUANTIZE` whether to run the quantized weights or not, in this case set to `fp8` as the weights at `meta-llama/Meta-Llama-3.1-405B-Instruct-FP8` are quantized to FP8 to allow faster inference and to fit within a node with 8 x H100s.

Additionally, as a recommendation you should also define `HF_HUB_ENABLE_HF_TRANSFER=1` to enable a faster download speed via the `hf_transfer` utility, as Llama 3.1 405B is around 400 GiB and downloading the weights may take longer otherwise.

Then you can already register the model within Vertex AI's Model Registry via the `google-cloud-aiplatform` Python SDK as follows:

In [ ]:
model = aiplatform.Model.upload(
    display_name="meta-llama--Meta-Llama-3.1-405B-Instruct-FP8",
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204",
    serving_container_environment_variables={
        "MODEL_ID": "meta-llama/Meta-Llama-3.1-405B-Instruct-FP8",
        "REVISION": "7c365a34d0fed5b4afb118cf253395f47f6efa72",
        "HUGGING_FACE_HUB_TOKEN": HF_TOKEN,
        "HF_HUB_ENABLE_HF_TRANSFER": "1",
        "NUM_SHARD": "8",
        "QUANTIZE": "fp8",
    },
)
model.wait()

![Llama 3.1 405B registered in Vertex AI](./imgs/vertex-ai-model.png)

## 3. Deploy Llama 3.1 405B on Vertex AI

Once Llama 3.1 405B is registered on Vertex AI Model Registry, then you can already call the `deploy` method to deploy a Vertex AI Endpoint that deploys Llama 3.1 405B in the Hugging Face DLC for TGI.

As mentioned before, since Llama 3.1 405B in FP8 occupies ~400 GiB, that means we need at least 400 GiB of GPU VRAM to load the model in a GPU node, and the GPUs within the node need to support the FP8 data type; so you will need to use a node with 8 x H100s, as each H100 comes with 80 GiB of VRAM, leading to a total of ~640 GiB of VRAM. So on, ~640 GiB is enough space to load Llama 3.1 405B in FP8 and to also fit the KV Cache and the CUDA Graphs, as it has ~200 GiB of free memory after loading the model.

In [ ]:
deployed_model = model.deploy(
    endpoint=aiplatform.Endpoint.create(display_name="Meta-Llama-3.1-405B-Endpoint"),
    machine_type="a3-highgpu-8g",
    accelerator_type="NVIDIA_H100_80GB",
    accelerator_count=8,
)

> Note that the Llama 3.1 405B deployment on Vertex AI may take around 30 minutes to deploy, as it needs to allocate the resources on Google Cloud, and then download the weights from the Hugging Face Hub and load those for inference too (the later taking ~15 minutes, while the rest will be the allocation time on Google Cloud).

![Llama 3.1 405B deployed in Vertex AI](./imgs/vertex-ai-endpoint.png)

## 4. Run online predictions with Llama 3.1 405B

Google Cloud Vertex AI will expose the endpoint `/predict` that is built on top of the `/vertex` endpoint within the Text Generation Inference (TGI) DLC, which runs the standard `/generate` method from TGI, but making sure that the I/O data is compliant with Vertex AI.

As `/generate` is the endpoint that is being exposed, you will need to format the messages with the chat template before sending the request to Vertex AI, so you will need to install 🤗`transformers` to use the `apply_chat_template` method from the `PreTrainedTokenizerFast`.

In [ ]:
%%bash
pip install --upgrade --quiet transformers

In [ ]:
import os
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3.1-405B-Instruct-FP8",
    revision="7c365a34d0fed5b4afb118cf253395f47f6efa72",
    token=HF_TOKEN,
)

In [ ]:
messages = [
    {"role": "system", "content": "You are an assistant that responds as a pirate."},
    {"role": "user", "content": "What's the Theory of Relativity?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

So that now you have a string out of the initial conversation messages, formatted using the default chat template for Llama 3.1, so that the above produces:

```text
<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an assistant that responds as a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat's the Theory of Relativity?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
```

Which is what you will be sending within the payload to the deployed Vertex AI Endpoint, as well as the generation arguments as in [Consuming Text Generation Inference (TGI) -> Generate](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/inference_client#huggingface_hub.InferenceClient.text_generation).

### 4.1 Via Python

### 4.1.1 Within the same session

If you are willing to run the online prediction within the current session i.e. the same one as the one used to deploy the model, you can send requests programatically via the `aiplatform.Endpoint` returned as of the `aiplatform.Model.deploy` method as in the following snippet.

In [ ]:
output = deployed_model.predict(
    instances=[
        {
            "inputs": inputs,
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ]
)

In [ ]:
output

Producing the following `output`:

```text

```

### 4.1.2 From a different session

If the Vertex AI Endpoint was deployed in a different session and you just want to use it, but don't have access to the `deployed_model` variable returned by the `aiplatform.Model.deploy` method, then you can also run the following snippet.

But note that you will need to know the name of the Vertex AI Endpoint in advance (if not overriden it should the the defined `display_name` for the model plus `_endpoint` as a suffix) i.e. `meta-llama--Meta-Llama-3.1-405B-Instruct-FP8_endpoint` in this case.

In [ ]:
import os
from google.cloud import aiplatform

aiplatform.init(project=os.getenv("PROJECT_ID"), location=os.getenv("LOCATION"))

endpoint = aiplatform.Endpoint("meta-llama--Meta-Llama-3.1-405B-Instruct-FP8_endpoint")
output = endpoint.predict(
    instances=[
        {
            "inputs": inputs,
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ],
)

In [ ]:
output

Producing the following `output`:

```text

```

## 4.2 Via the Vertex AI Online Prediction UI

...

![Llama 3.1 405B online prediction in Vertex AI](./imgs/vertex-ai-online-prediction.png)

## 5. Clean up resources

Finally, you can already release the resources that you've created as follows, to avoid unnecessary costs:
* `deployed_model.undeploy_all` to undeploy the model from all the endpoints.
* `deployed_model.delete` to delete the endpoint/s where the model was deployed gracefully, after the `undeploy_all` method.
* `model.delete` to delete the model from the registry.

In [ ]:
# deployed_model.undeploy_all()
# deployed_model.delete()
# model.delete()

Alternatively, you can also remove those from the Google Cloud Console following the steps:
* Go to Vertex AI in Google Cloud
* Go to Deploy and use -> Online prediction
* Click on the endpoint and then on the deployed model/s to "Undeploy model from endpoint"
* Then go back to the endpoint list and remove the endpoint
* Finally, go to Deploy and use -> Model Registry, and remove the model

## Conclusion

And that's it! You have already registered and deployed Llama 3.1 405B on Google Cloud Vertex AI, then ran online prediction both programatically and via the Google Cloud Console, and finally cleaned up the resources used to avoid unnecessary costs.

Thanks to the Hugging Face DLCs, Text Generation Inference (TGI), and Google Cloud Vertex AI, deploying a high-performance text generation container for serving Large Language Models (LLMs) has never been easier. And we’re not going to stop here – stay tuned as we enable more experiences to build AI with open models on Google Cloud!